### Die Mandelbrotmenge

Für jede Zahl c definieren wir rekursiv eine Folge von Zahlen wie folgt:

$$z_n =\begin{cases}
 0   & n = 0 \\ 
 z_{n-1}^2 + c & n > 0 
\end{cases} $$

In [2]:
def mandel(c,n):
    if n == 0: return 0
    return mandel(c,n-1)**2 + c

Für manche c wächst die Folge ins Unendliche (mal früher, mal später), für andere c scheint die Folge beschränkt zu bleiben. 

In [ ]:
c = 0.25
#c = 0.26
#c = 0.27
#c = 0.28

for n in range(100):
    print(mandel(c,n))

Es gilt: Wenn ein Folgenglied betragsmäßig größer als 2 wird, dann explodiert die Folge.

Um zu entscheiden, ob eine Zahl c zur Mandelbrotmenge gehört, gehen wir wie folgt vor.
 -  Wir legen eine Rekursionstiefe fest, z.B. 17.
 -  Wir bilden die zu c gehörende Folge bis zum Index n=17.
 -  Wenn keines der Folgenelemente $z_n$ den Betrag > 2 hat, dann gehört c zur Mandelbrotmenge, sonst nicht.
 

In [34]:
c = 0.27
def inMandel(c):
    for i in range(18):
        if abs(mandel(c,i)) > 2:
            return False
    return True

for i in range(10):
    print(c,inMandel(c))
    c += 0.001

0.27 True
0.271 True
0.272 True
0.273 True
0.274 True
0.275 True
0.276 True
0.277 False
0.278 False
0.279 False


Je höher wir die Rekursionstiefe wählen, desto kleiner wird die Mandelbrotmenge.

Wir färben die Punkte auf der Zahlengerade, die zur Mandelbrotmenge gehören schwarz, die anderen weiß.

#### Färbung von Punkten in der Ebene

Wir interpretieren die Punkte in der Ebene als komplexe Zahlen. Dadurch ergibt sich:

- Die Addition zweier Punkte ergibt sich durch Vektoraddition
- Wird ein Punkt quadriert, so wird sein Betrag (= Abstand zum Ursprung) quadriert und der Winkel verdoppelt. 



In [5]:
c = -0.2 + 0.5j
for n in range(17):
    z = mandel(c,n)
    print(z,abs(z))

0 0
(-0.2+0.5j) 0.5385164807134505
(-0.41000000000000003+0.3j) 0.5080354318352215
(-0.12189999999999998+0.254j) 0.28173677431247773
(-0.24965639+0.4380748j) 0.504220034769417
(-0.3295812173272079+0.28126365376405604j) 0.4332816886779212
(-0.17048546411382257+0.3146015652050882j) 0.3578259888599441
(-0.2699088513553859+0.39273001229015103j) 0.4765371450300658
(-0.2813860745134384+0.28799738697995747j) 0.4026420467827046
(-0.2037643719772011+0.33792309161516426j) 0.3946035163730981
(-0.27267209655948754+0.36228662692088376j) 0.4534332059828196
(-0.2569015278035651+0.3024290917640355j) 0.396814503934805
(-0.22546496055741344+0.3446110085471494j) 0.4118145160761571
(-0.26792229877272744+0.344604585100533j) 0.4365027814939707
(-0.24696996189264786+0.3153454947844865j) 0.4005458065666504
(-0.23844861900361666+0.34423827034011434j) 0.41875736491611176
(-0.2616422428620217+0.33583371965841213j) 0.4257240309278701


In [6]:
import numpy as np
matplotlib inline


def mandelbrot_set(xmin, xmax, ymin, ymax, xn, yn, maxiter, horizon=2.0):
    X = np.linspace(xmin, xmax, xn, dtype=np.float32)
    Y = np.linspace(ymin, ymax, yn, dtype=np.float32)
    C = X + Y[:, None]*1j
    N = np.zeros(C.shape, dtype=int)
    Z = np.zeros(C.shape, np.complex64)
    for n in range(maxiter):
        I = np.less(abs(Z), horizon)
        N[I] = n
        Z[I] = Z[I]**2 + C[I]
    N[N == maxiter-1] = 0
    return Z, N


if __name__ == '__main__':
    import time
    import matplotlib
    from matplotlib import colors
    import matplotlib.pyplot as plt

    xmin, xmax, xn = -2.25, +0.75, 3000/2
    ymin, ymax, yn = -1.25, +1.25, 2500/2
    maxiter = 200
    horizon = 2.0 ** 40
    log_horizon = np.log(np.log(horizon))/np.log(2)
    Z, N = mandelbrot_set(xmin, xmax, ymin, ymax, xn, yn, maxiter, horizon)

    # Normalized recount as explained in:
    # https://linas.org/art-gallery/escape/smooth.html
    # https://www.ibm.com/developerworks/community/blogs/jfp/entry/My_Christmas_Gift

    # This line will generate warnings for null values but it is faster to
    # process them afterwards using the nan_to_num
    with np.errstate(invalid='ignore'):
        M = np.nan_to_num(N + 1 -
                          np.log(np.log(abs(Z)))/np.log(2) +
                          log_horizon)

    dpi = 72
    width = 10
    height = 10*yn/xn
    fig = plt.figure(figsize=(width, height), dpi=dpi)
    ax = fig.add_axes([0.0, 0.0, 1.0, 1.0], frameon=False, aspect=1)

    # Shaded rendering
    light = colors.LightSource(azdeg=315, altdeg=10)
    M = light.shade(M, cmap=plt.cm.hot, vert_exag=1.5,
                    norm=colors.PowerNorm(0.3), blend_mode='hsv')
    plt.imshow(M, extent=[xmin, xmax, ymin, ymax], interpolation="bicubic")
    ax.set_xticks([])
    ax.set_yticks([])

    # Some advertisement for matplotlib
    year = time.strftime("%Y")
    major, minor, micro = matplotlib.__version__.split('.', 2)
    text = ("The Mandelbrot fractal set\n"
            "Rendered with matplotlib %s.%s, %s - http://matplotlib.org"
            % (major, minor, year))
    ax.text(xmin+.025, ymin+.025, text, color="white", fontsize=12, alpha=0.5)


C:\Users\khthe\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """
C:\Users\khthe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  
